# Image and Video Processing Lab
<center>
<h2>Mini Project - JPEG Compression</h2>



---

<h3> Submitted by:<h3>
<h4> Sneha Dash (21EC39023), Unnati Singh (21EC39027) </h4></center>

___

### Importing the required packages:

In [1]:
import os
import numpy as np
import cv2
import pickle
import gzip

from collections import Counter
from itertools import chain

## JPEG Compression Pipeline:

### Reading Image in RGB format:

In [2]:
def read_image(imagepath):
    image_bgr = cv2.imread(imagepath)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    return image_rgb

### Colour Space conversion to YCbCr

The following function converts an RGB image to the YCbCr color space using a standard transformation matrix. This conversion is commonly used in image and video compression.

- **Parameters**:
  - **image**: A NumPy array representing the RGB image, with pixel values in the range [0, 255].

- **Returns**:
  - **ycbcr_image**: A NumPy array representing the image in YCbCr color space, with pixel values clipped to the range [0, 255] and cast to `uint8`.

**Process**:

The function uses the following matrix transformation to convert RGB to YCbCr:

$$
\begin{bmatrix}
Y \\
Cb \\
Cr
\end{bmatrix}
=
\begin{bmatrix}
0.299 & 0.587 & 0.114 \\
-0.168736 & -0.331264 & 0.5 \\
0.5 & -0.418688 & -0.081312
\end{bmatrix}
\begin{bmatrix}
R \\
G \\
B
\end{bmatrix}
+
\begin{bmatrix}
0 \\
128 \\
128
\end{bmatrix}
$$

This formula adjusts the RGB channels to separate luminance (Y) and chrominance (Cb, Cr) components.

The result is clipped to ensure the values remain within the valid range for image data [0, 255].


In [3]:
def rgb_to_ycbcr(image):
    conversion_matrix = np.array([[ 0.299,  0.587,  0.114],
                                  [-0.168736, -0.331264,  0.5],
                                  [ 0.5, -0.418688, -0.081312]])

    offset = np.array([0, 128, 128])
    ycbcr_image = np.dot(image, conversion_matrix.T) + offset

    return np.clip(ycbcr_image, 0, 255).astype(np.uint8)

### Subsampling:

This function performs chroma subsampling on a YCbCr image by downsampling the chrominance channels (Cb and Cr) based on specified horizontal and vertical ratios. This is often used in image and video compression to reduce the amount of chrominance data while maintaining luminance (Y) quality.

- **Parameters**:
  - **ycbcr_image**: A NumPy array representing the YCbCr image.
  - **ratio1h**, **ratio1v**: Horizontal and vertical subsampling ratios for the Cb (blue-difference) channel.
  - **ratio2h**, **ratio2v**: Horizontal and vertical subsampling ratios for the Cr (red-difference) channel.

- **Returns**:
  - **Y**: The luminance channel (Y), unchanged.
  - **Cb**: The subsampled blue-difference channel (Cb).
  - **Cr**: The subsampled red-difference channel (Cr).

**Process**:
The function slices the Cb and Cr channels based on the specified horizontal and vertical subsampling ratios, reducing their resolution, while the luminance channel (Y) is retained at full resolution.


In [4]:
def chroma_subsampling(ycbcr_image, ratio1h = 2, ratio1v = 2, ratio2h = 2, ratio2v = 2):
    Y = ycbcr_image[:, :, 0]
    Cb = ycbcr_image[::ratio1h, ::ratio1v, 1]
    Cr = ycbcr_image[::ratio2h, ::ratio2v, 2]

    return Y, Cb, Cr

### Formation of 8×8 blocks for processing:

This function divides a single-channel image (such as Y, Cb, or Cr) into non-overlapping square blocks of a specified size (default is 8x8). This step is commonly used in image compression algorithms like JPEG.

- **Parameters**:
  - **channel**: A 2D NumPy array representing a single-channel image (e.g., luminance or chrominance).
  - **block_size**: The size of each block (default is 8).

- **Returns**:
  - **blocks**: A 3D NumPy array where each slice along the first dimension is a flattened `block_size x block_size` block from the original image.

**Process**:
1. The function ensures that the image dimensions are divisible by the block size by trimming any extra pixels.
2. The image is then reshaped into smaller blocks, where each block is a non-overlapping sub-region of size `block_size x block_size`.
3. The blocks are flattened and returned as a 3D array of shape `(num_blocks, block_size, block_size)`.


In [5]:
def block_formation(channel, block_size=8):
    h, w = channel.shape
    h -= h % block_size
    w -= w % block_size
    channel = channel[:h, :w]

    blocks = (channel.reshape(h // block_size, block_size, -1, block_size).swapaxes(1, 2).reshape(-1, block_size, block_size))

    return blocks

### Performing DCT on channel blocks:

We apply the Discrete Cosine Transform (DCT) to a given square block, commonly used in image compression to convert spatial data into frequency components.

- **Parameters**:
  - **block**: A 2D NumPy array representing an $N \times N$ block of pixel values.

- **Returns**:
  - **dct_matrix**: A 2D NumPy array of DCT coefficients for the input block.

The DCT formula applied here is:

$$
DCT(u, v) = \frac{1}{4} \, C(u) C(v) \sum_{x=0}^{N-1} \sum_{y=0}^{N-1} \text{block}[x, y] \cos \left(\frac{(2x + 1) u \pi}{2N}\right) \cos \left(\frac{(2y + 1) v \pi}{2N}\right)
$$

where $C(u) = \frac{1}{\sqrt{2}}$ if $u = 0$ (and similarly for $C(v)$), otherwise $C(u) = 1$.


In [6]:
def dct_block(block):
    N = block.shape[0]

    u = np.arange(N)
    v = np.arange(N)

    cos_u = np.cos((2 * np.outer(np.arange(N), u) + 1) * np.pi / (2 * N))
    cos_v = np.cos((2 * np.outer(np.arange(N), v) + 1) * np.pi / (2 * N))

    cu = np.where(u == 0, 1 / np.sqrt(2), 1)
    cv = np.where(v == 0, 1 / np.sqrt(2), 1)

    dct_matrix = 0.25 * (cu[:, None] * cv[None, :] * np.sum(block * cos_u[:, :, None] * cos_v[:, None, :], axis=(0, 1)))

    return dct_matrix

### Quantising:

Then we quantize a DCT-transformed block by dividing each element by a corresponding element in a quantization matrix, then rounding to the nearest integer. This step is crucial in image compression to reduce precision and data size.

- **Parameters**:
  - **block**: A 2D NumPy array representing a DCT-transformed block.
  - **quant_matrix**: A 2D NumPy array (of the same size as `block`) used for quantization.

- **Returns**:
  - A 2D NumPy array of quantized DCT coefficients as integers.
  
The quantization formula is:

$$
\mathrm{quantized\_block} = \mathrm{round}\left(\frac{\mathrm{block}}{\mathrm{quant\_matrix}}\right)
$$


In [7]:
def quantize(block, quant_matrix):
    return np.round(block / quant_matrix).astype(int)

### Run Length encoding:

This function performs run-length encoding (RLE) on a DCT-transformed block. The function first arranges the block's elements in a zigzag order to prioritize lower-frequency components, then encodes sequences of zeros to achieve data compression.

- **Parameters**:
  - **block**: A 2D NumPy array representing a quantized DCT block.

- **Returns**:
  - **rle**: A list where non-zero values are directly listed, and sequences of zeros are represented as tuples `(zeros, value)`, where `zeros` is the count of consecutive zeros before `value`.

### Process:

1. **Zigzag Scan**: The block's elements are reordered in a zigzag pattern to prioritize lower-frequency components (top-left to bottom-right, following a diagonal pattern).
   
   The zigzag pattern is defined as:

   $$
   \text{zigzag\_order} = \left[ (0, 0), (0, 1), (1, 0), (2, 0), (1, 1), (0, 2), \dots \right]
   $$

2. **Run-Length Encoding**:
   - Sequences of consecutive zeros are encoded as a tuple `(number of zeros, next non-zero value)`.
   - Non-zero values are stored as they are.

   Example:
   - For a zigzag scan like `[0, 0, 1, 0, 0, 2]`, the RLE output would be `[(2, 1), (2, 2)]`.

This encoding helps compress the data by reducing the storage needed for long sequences of zeros.


In [8]:
def run_length_encode(block):

    def zigzag_scan(block):
        zigzag_order = [
            (0, 0), (0, 1), (1, 0), (2, 0), (1, 1), (0, 2), (0, 3), (1, 2), (2, 1), (3, 0),
            (3, 1), (2, 2), (1, 3), (0, 4), (0, 5), (1, 4), (2, 3), (3, 2), (4, 0), (5, 0),
            (4, 1), (3, 3), (2, 4), (1, 5), (0, 6), (0, 7), (1, 6), (2, 5), (3, 4), (4, 3),
            (5, 1), (6, 0), (7, 0), (6, 1), (5, 2), (4, 4), (3, 5), (2, 6), (1, 7), (0, 7),
            (1, 6), (2, 5), (3, 4), (4, 3), (5, 1), (6, 0), (7, 0), (6, 1), (5, 2), (4, 4),
            (3, 5), (2, 6), (1, 7), (0, 7)
        ]
        return [block[x, y] for x, y in zigzag_order]

    zigzag_values = zigzag_scan(block)

    rle = []
    zeros = 0
    for value in zigzag_values:
        if value == 0:
            zeros += 1
        else:
            if zeros > 0:
                rle.append((zeros, value))
                zeros = 0
            else:
                rle.append(value)
    return rle

### Huffman Encoding:

Finally, we perform Huffman coding on the input data, which is typically a run-length encoded (RLE) sequence. The function builds a Huffman tree based on symbol frequencies, then generates the corresponding binary codes for each symbol to compress the data.

- **Parameters**:
  - **rle_data**: A list of symbols (e.g., from run-length encoding) that need to be compressed using Huffman coding.

- **Returns**:
  - **codes**: A dictionary mapping each symbol to its Huffman binary code.
  - **encoded_sequence**: A string representing the input data encoded using the generated Huffman codes.

### Process:

1. **Frequency Calculation**: The function first calculates the frequency of each symbol in the input `rle_data` using the `Counter` from the `collections` module.
   
2. **Tree Construction**: The Huffman tree is built using the following steps:
   - Sort the symbols by frequency.
   - Combine the two least frequent symbols into a new node, and repeat until only one node remains.

3. **Huffman Code Generation**: Starting from the root of the Huffman tree, assign binary codes:
   - Traverse left with '0' and right with '1' to generate codes for each symbol.

4. **Encoding**: The input sequence is then encoded by replacing each symbol with its corresponding Huffman code.

This method efficiently compresses data by assigning shorter codes to more frequent symbols and longer codes to less frequent symbols.



In [9]:
def huffman(rle_data):
    symbol_frequencies = Counter(rle_data)
    nodes = [(symbol, freq) for symbol, freq in symbol_frequencies.items()]

    def make_tree(nodes):
        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)
        while len(nodes) > 1:
            key1, c1 = nodes.pop()
            key2, c2 = nodes.pop()
            node = (key1, key2)
            nodes.append((node, c1 + c2))
            nodes = sorted(nodes, key=lambda x: x[1], reverse=True)
        return nodes[0][0]

    def huffman_code_tree(node, bin_string=''):
        if isinstance(node, tuple):
            left, right = node
            left_code = huffman_code_tree(left, bin_string + '0')
            right_code = huffman_code_tree(right, bin_string + '1')
            return {**left_code, **right_code}
        else:
            return {node: bin_string}

    tree = make_tree(nodes)
    codes = huffman_code_tree(tree)

    for symbol in set(rle_data):
        if symbol not in codes:
            codes[symbol] = '0'

    encoded_sequence = ''.join(codes[symbol] for symbol in rle_data)

    return codes, encoded_sequence


___

## JPEG Compression:

The following function uses all the above defined functions to compress the given image and then saves them as a pickle file, giving a comparison between the sizes of the original image and the saved compressed file.

In [10]:
def compress_save(image_path):

    def compression(image_path, quality=50):
        image_rgb = read_image(image_path)
        ycbcr_image = rgb_to_ycbcr(image_rgb)

        Y, Cb, Cr = chroma_subsampling(ycbcr_image)

        quant_matrix_y = np.array([
            [16, 11, 10, 16, 24, 40, 51, 61],
            [12, 12, 14, 19, 26, 58, 60, 55],
            [14, 13, 16, 24, 40, 57, 69, 56],
            [14, 17, 22, 29, 51, 87, 80, 62],
            [18, 22, 37, 56, 68, 109, 103, 77],
            [24, 35, 55, 64, 81, 104, 113, 92],
            [49, 64, 78, 87, 103, 121, 120, 101],
            [72, 92, 95, 98, 112, 100, 103, 99]
        ]) * (100 - quality) / 50

        quant_matrix_cbcr = quant_matrix_y

        def process_channel(channel, quant_matrix):
            blocks = block_formation(channel)
            compressed_blocks = []
            for block in blocks:
                dct_block_result = dct_block(block)
                quantized_block = quantize(dct_block_result, quant_matrix)
                rle_encoded_block = run_length_encode(quantized_block)
                compressed_blocks.append(rle_encoded_block)
            return compressed_blocks

        compressed_Y = process_channel(Y, quant_matrix_y)
        compressed_Cb = process_channel(Cb, quant_matrix_cbcr)
        compressed_Cr = process_channel(Cr, quant_matrix_cbcr)

        flat_compressed_Y = list(chain.from_iterable(compressed_Y))
        flat_compressed_Cb = list(chain.from_iterable(compressed_Cb))
        flat_compressed_Cr = list(chain.from_iterable(compressed_Cr))

        _, huffman_Y = huffman(flat_compressed_Y)
        _, huffman_Cb = huffman(flat_compressed_Cb)
        _, huffman_Cr = huffman(flat_compressed_Cr)

        complete_data = {
            'Y': flat_compressed_Y,
            'Cb': flat_compressed_Cb,
            'Cr': flat_compressed_Cr,
            'huffman_Y': huffman_Y,
            'huffman_Cb': huffman_Cb,
            'huffman_Cr': huffman_Cr,
        }

        compressed_data = { 'huffman_Y': huffman_Y,
                            'huffman_Cb': huffman_Cb,
                            'huffman_Cr': huffman_Cr,
                            }

        return complete_data, compressed_data

    if not os.path.exists('output'):
        os.makedirs('output')

    pickle_file_path = "output/" + os.path.splitext(os.path.basename(image_path))[0] + '_bin' + '.pkl.gz'
    picklepath2 = "output/encoded_only" + os.path.splitext(os.path.basename(image_path))[0] + '_bin' + '.pkl.gz'
    binary, comp = compression(image_path)

    with gzip.open(pickle_file_path, 'wb') as file:
        pickle.dump(binary, file)

    with gzip.open(picklepath2, 'wb') as file:
        pickle.dump(comp, file)

    input_image_size = os.path.getsize(image_path)
    compressed_file_size = os.path.getsize(picklepath2)

    print(f"Compressed file saved at: {pickle_file_path}")
    print(f"Original Size: {input_image_size/ (1024 ** 2):.2f} MB")
    print(f"Compressed Size: {compressed_file_size/ (1024 ** 2):.2f} MB")

    return binary

## Running the final defined function:

In [11]:
comp = compress_save(input('Enter image path: '))

Compressed file saved at: output/green_bin.pkl.gz
Original Size: 1.94 MB
Compressed Size: 0.33 MB


In [12]:
for key in comp:
    print(f"{key}: {comp[key][:200]}")

Y: [11, 14, 20, 17, 18, 18, 11, 18, 17, 17, 13, 16, 13, 8, 5, 10, 10, 11, 13, 10, 10, 9, 7, 5, 4, 3, 5, 5, 5, 4, 6, 5, 3, 3, 5, 4, 3, 4, 5, 3, 5, 5, 5, 4, 6, 5, 3, 3, 5, 4, 3, 4, 5, 3, 11, 14, 20, 17, 18, 18, 11, 18, 17, 17, 13, 16, 13, 8, 5, 10, 10, 11, 13, 10, 10, 9, 7, 5, 4, 3, 5, 5, 5, 4, 6, 5, 3, 3, 5, 4, 3, 4, 5, 3, 5, 5, 5, 4, 6, 5, 3, 3, 5, 4, 3, 4, 5, 3, 11, 14, 20, 17, 18, 17, 11, 18, 16, 17, 13, 15, 13, 8, 5, 10, 10, 11, 13, 10, 10, 8, 7, 4, 4, 3, 5, 5, 5, 4, 6, 5, 3, 3, 4, 4, 3, 4, 5, 3, 5, 5, 5, 4, 6, 5, 3, 3, 4, 4, 3, 4, 5, 3, 11, 14, 20, 17, 18, 17, 11, 18, 16, 17, 13, 15, 13, 8, 5, 10, 10, 11, 13, 10, 10, 9, 7, 4, 4, 3, 5, 5, 5, 4, 6, 5, 3, 3, 4, 4, 3, 4]
Cb: [4, 5, 8, 7, 7, 7, 4, 7, 6, 7, 5, 6, 5, 3, 2, 4, 4, 4, 5, 4, 4, 3, 3, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 4, 5, 8, 7, 7, 7, 4, 7, 6, 7, 5, 6, 5, 3, 2, 4, 4, 4, 5, 4, 4, 3, 3, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 